In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers

In [40]:
#載入mnist資料集
(x, y), (x_val, y_val) = datasets.mnist.load_data()

In [41]:
# Convert the value from [0,255] to [0,1]
# 將資料集型態從numpy變為tensor，加速運算
x = tf.convert_to_tensor(x, dtype=tf.float32)/255.
y = tf.convert_to_tensor(y, dtype=tf.int32)

# 利用one_hot將0-9轉為機率[1,0,0,0,0,0,0,0,0,0]
y = tf.one_hot(y, depth=10)

print(x.shape, y.shape)

(60000, 28, 28) (60000, 10)


In [42]:
# 200 pictures are loaded at once
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_dataset = train_dataset.batch(200)
print(train_dataset)

<BatchDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.float32, tf.float32)>


In [43]:
# 建構出3-hidden-layer模型
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

# 設定optimizer-SGD(Stochastic gradient descent)
opt =optimizers.SGD(lr=0.01)

In [44]:
# Training & update the parameter of the model
# epoch: 訓練完 所有batch 即為 1個 epoch
def train_epoch(epoch):
    
    # 訓練完 1個 batch 即為 1個 step
    for step, (x, y) in enumerate(train_dataset):

        # 利用tf.GradientTape()建立feed-forward的路徑
        with tf.GradientTape() as tape:
            
            # 將資料從 (200, 28, 28) => (200, 1, 756)
            x = tf.reshape(x,(-1,28*28))
            
            # output值為上面 x輸入到模型產生的結果
            out = model(x)
            
            # 建立loss函數
            loss = tf.reduce_sum(tf.square(out-y)) / x.shape[0]
  
        # 計算出 gradient值，此值為loss function以及[w11,b11,w12, b12,...]
        grads = tape.gradient(loss, model.trainable_variables)
        
        # 將 gradient值與[w11,b11,w12, b12,...]相成積後調整原有參數
        opt.apply_gradients(zip(grads, model.trainable_variables))

        # 每 100個 step就印出loss結果
        if step %100 ==0:
            print(epoch, step, "loss:", loss.numpy())

In [45]:
# 確保是目前在此.py檔才會執行
if __name__ == '__main__':
    for epoch in range(20):
            train_epoch(epoch)

0 0 loss: 1.892961
0 100 loss: 0.57648474
0 200 loss: 0.4179177
1 0 loss: 0.3575475
1 100 loss: 0.38233688
1 200 loss: 0.31255773


KeyboardInterrupt: 